__Imports__

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from nilearn.image import load_img
from nilearn.plotting import find_parcellation_cut_coords

import pandas as pd

In [ ]:

def FileToMatrix(fileName):
    '''
    Function to load a particular time series, and generate a matrix for it!
    '''
    data = np.loadtxt(fileName)
    correlationMatrix = np.corrcoef(data.T) # need to transpose (I think for all?)
    np.fill_diagonal(correlationMatrix, 0) # remove leading zeros
    
    return correlationMatrix
    

In [ ]:
def LoadAtlas(fileName):
    '''
    Simple function to load an atlas, and return coords.
    '''
    
    atlasImg = load_img(fileName)
    coords = find_parcellation_cut_coords(atlasImg)
    return coords

In [ ]:
def PercentageThresholding(correlationMatrix, percentageToKeep):
    '''
    Function to apply a percentage based thresholding, returns new adjacency matrix
    '''
    
    # Flatten the matrix, sort the values in descending order
    sortedValues = np.sort(correlationMatrix.flatten())[::-1]
    
    # Determine the thresholded value based on percentage (how many elements to keep)
    numElementsToKeep = int(len(sortedValues) * percentageToKeep)
    thresholdValue = sortedValues[numElementsToKeep - 1]
    
    # Create a new adjacency matrix based on our thresholding
    adjMatrix = np.zeros(correlationMatrix.shape, dtype=int) # int for binary
    adjMatrix[correlationMatrix >= thresholdValue] = 1 # 1 if keeping, 0 if thresholded
    
    return adjMatrix
    
    

In [ ]:
def OutputBrainNetViewer(nodeData, G, coords, outputNodeName, outputEdgeName, weighted=False):
    '''
    Formats data for Brain Net Viewer.
    '''
    # Add coordinates from atlas
    
    if weighted:
        weightedDegrees = dict(G.degree(weight='weight'))
    else:
        weightedDegrees = dict(G.degree())
    

    nodeData['roi'] = nodeData.index
    
    nodeData['x'] = [coord[0] for coord in coords]
    nodeData['y'] = [coord[1] for coord in coords]
    nodeData['z'] = [coord[2] for coord in coords]
    
    nodeData['weightedDegree'] = nodeData.index.map(weightedDegrees)

    # Save node file for Brain Net Viewer
    nodeData[['x', 'y', 'z', 'roi', 'weightedDegree']].to_csv(outputNodeName, sep='\t', index=False, header=False)

    # Save adjacency matrix as .edge file
    edge_matrix = nx.to_numpy_array(G)
    np.savetxt(outputEdgeName, edge_matrix)